<a href="https://www.kaggle.com/code/tolawuwo/unemployment-vs-participation-rate-in-the-us?scriptVersionId=172199507" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction
This is a brief Exploratory Data Analysis(EDA) to explore the relationship between unemployment and labor force participation in the US. Unemployment refers to the percentage of people who are actively seeking employment but are currently without a job. Labor force participation refers to the percentage of the working-age population that is either employed or actively seeking employment.

## Installing Fred API package

In [ ]:
!pip install fredapi > /dev/null

## Install the neccessary package required for the EDA 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import time

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

from fredapi import Fred

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
fred_key = secrets.get_secret('fred-api')

### Create the Fred Object

In [ ]:
fred = Fred(api_key=fred_key)

### Browse the data for "Unemployment rate per states United States"

Search through the dataset to extract unemployment rate data for each state.

In [ ]:
unemp_df = fred.search('unemployment rate by state', filter=('frequency','Monthly'))
unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemp_df = unemp_df.loc[unemp_df['title'].str.contains("Unemployment Rate in")]

### Check the data
Basic data quality checks...quick glimpse of the data

In [ ]:
unemp_df.info()

### Preparing the data for Unemployment rate per state

In [ ]:
all_results = []

for myid in unemp_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
    time.sleep(0.1) # Slowing doing the API request
unemp_states = pd.concat(all_results, axis=1)\
    .drop((['LASMT261982000000003', 
            'LASMT391746000000003', 
            'PRUR', 
            'CMWRUR', 
            'CSOUUR', 
            'CNERUR', 
            'CWSTUR', 
            'CNEWUR']), 
           axis=1)

Replace the id of the states with the full state names as seen in the "title" column.

In [ ]:
id_to_states = unemp_df['title'].str.replace('Unemployment Rate in ', '').to_dict()
unemp_states.columns = [id_to_states[c] for c in unemp_states.columns]
unemp_states = unemp_states.rename(columns={"the District of Columbia" : "District Of Columbia"})

### Plot unemployment rate of all the states.

In [ ]:
fig = px.line(unemp_states)
fig.show()

### Plot of the unployment rate in April 2020

In [ ]:
ax = unemp_states.loc[unemp_states.index == '2020-04-01'].T \
    .sort_values('2020-04-01') \
    .plot(kind='barh', figsize=(8, 12),
         title= "Unemployment in the US by States, April 2020")
ax.legend().remove()
plt.show()

Upon closer examination of the drop around April 2020, it can be observed that the COVID-19 pandemic significantly impacted the unemployment rate. Nevada experienced the most pronounced impact, whereas Wyoming was the least affected.

## Browse the Fred dataset for "Participation rate" per states United States"

In [ ]:
# Participation rate
part_df = fred.search('participation rate by state', filter=('frequency','Monthly'))
part_df = part_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
part_df = part_df.loc[part_df['title'].str.contains("Labor Force Participation Rate for")]

In [ ]:
part_df.info()

In [ ]:
part_all_results = []

for myid in part_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    part_all_results.append(results)
part_states = pd.concat(part_all_results, axis=1)

part_id_to_states = part_df['title'].str.replace('Labor Force Participation Rate for ', '').to_dict()
part_states.columns = [part_id_to_states[c] for c in part_states.columns]

In [ ]:
fig = px.line(part_states)
fig.show()

In [ ]:
ax = part_states.loc[part_states.index == '2020-04-01'].T \
    .sort_values('2020-04-01') \
    .plot(kind='barh', figsize=(8, 12),
         title= "Participation rate in the US by States, April 2020")
ax.legend().remove()
plt.show()

# Plot Unemployment vs Participation

### Plot for 50 states (excluding "District Of Columbia")

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(30, 30), sharex=True)
axs = axs.flatten()

i = 0
for state in unemp_states.columns:
    if state == "District Of Columbia":
        continue
    ax2 = axs[i].twinx()
    unemp_states.query('index >= 2020 and index <= 2024')[state] \
        .plot(ax=axs[i], label='Unemployment')
    part_states.query('index >= 2020 and index <= 2024')[state] \
        .plot(ax=ax2, label='Participation', color=color_pal[1])
    ax2.grid(False)
    axs[i].set_title(state, fontsize=8)
    i += 1
plt.tight_layout()
plt.show()

### Plot for "District Of Columbia"

In [ ]:
state_dc = 'District Of Columbia'
fig, ax = plt.subplots(figsize=(10, 5), sharex=True)
ax2 = ax.twinx()
unemp_states.query('index >= 2020 and index <= 2024')[state_dc] \
    .plot(ax=ax, label='Unemployment')
part_states.query('index >= 2020 and index <= 2024')[state_dc] \
    .plot(ax=ax2, label='Participation', color=color_pal[1])
ax2.grid(False)
ax.set_title(state_dc)
fig.legend(labels=['Unemployment','Participation'])
plt.show();

# Summary and Conclusion

In conclusion, the relationship between unemployment and labor force participation measures can vary based on economic conditions and demographic factors. Generally, during periods of economic downturns or recessions, both the unemployment rate and the labor force participation rate may increase. The plot above is between 2020 and 2024 and we can see the relationship between the unemployment rate and the labor force participation rate particulary around April 2020, shortly after we many states annouced Covid-19 locked-down.

Please feel free to explore further and let me know your thoughts or questions.